In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [2]:
train2 = pd.read_csv("sets_de_datos/dima_train_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [3]:
id_zonas = pd.read_csv("sets_de_datos/train.csv")

In [4]:
id_zonas[['idzona','precio']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 2 columns):
idzona    211379 non-null float64
precio    240000 non-null float64
dtypes: float64(2)
memory usage: 3.7 MB


In [5]:
id_zonas.groupby('ciudad')['idzona'].count()

ciudad
Abalá                0
Abasolo              4
Abejones             0
Acajete              0
Acambay              3
                    ..
ácatlán de Juárez    1
ácaxochitlán         5
ácámbaro             2
áutlán de Navarro    1
áyotlán              1
Name: idzona, Length: 875, dtype: int64

In [6]:
zona_por_ciudad = id_zonas.groupby('ciudad').agg({'idzona':'mean'})
avg = zona_por_ciudad['idzona'].mean()

In [7]:
zona_por_ciudad = zona_por_ciudad.fillna(zona_por_ciudad['idzona'].mean())

In [8]:
info_ids = zona_por_ciudad['idzona'].to_dict()

In [10]:
train2['valor_zona'] = train2.apply(lambda x: info_ids.get(x['ciudad'], avg), axis = 1)

In [11]:
train2

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra,antiguedad_sobre_tipodepropiedad_mean_antiguedad,direccion_cantidad_al_menos_una_mayuscula,direccion_cantidad_fijos_top_k,titulo_cantidad_fijos_top_k,titulo_palabras_top_k_sobre_total_palabras,ciudad_distancia_al_origen,ciudad_mean_mean_todas,ciudad_mean_antiguedad_sobre_mean_metrocubiertos,valor_zona
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,0.0,2.0,1.0,...,0.336400,0.000000,3.0,6.0,2.0,0.200000,100.735420,1.400474,0.069995,2.345665e+07
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,0.292252,1.475504,2.0,3.0,5.0,0.400000,100.580048,1.651640,0.048673,2.604988e+04
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,0.137507,0.597851,2.0,2.0,8.0,0.500000,105.849480,1.280135,0.048689,7.974404e+04
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,0.132153,0.119570,3.0,5.0,12.0,0.090909,102.350349,1.515007,0.019753,7.970230e+04
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,0.297729,1.187935,3.0,2.0,3.0,0.000000,105.462814,1.634266,0.027120,1.996547e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,0.132153,0.000000,1.0,2.0,12.0,0.384615,102.350349,1.515007,0.019753,7.970230e+04
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.0,3.0,3.0,...,0.130846,0.000000,2.0,3.0,9.0,0.272727,101.225327,1.537764,0.037382,6.843710e+04
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,0.336400,2.375870,3.0,5.0,5.0,0.250000,100.735420,1.400474,0.069995,2.345665e+07
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,0.138500,2.391406,3.0,4.0,8.0,0.166667,100.846626,1.358463,0.102012,2.894259e+04


In [16]:
test = pd.read_csv("sets_de_datos/dima_test_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [17]:
test

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad,tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra,antiguedad_sobre_tipodepropiedad_mean_antiguedad,direccion_cantidad_al_menos_una_mayuscula,direccion_cantidad_fijos_top_k,titulo_cantidad_fijos_top_k,titulo_palabras_top_k_sobre_total_palabras,ciudad_distancia_al_origen,ciudad_mean_mean_todas,ciudad_mean_antiguedad_sobre_mean_metrocubiertos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,2.0,...,0.790404,0.105755,3.467539,2.0,4.0,9.0,0.222222,100.743263,1.431429,0.065100
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,0.0,1.0,1.0,...,0.667304,0.284267,0.000000,0.0,0.0,6.0,0.333333,91.761078,1.654162,0.009316
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,1.238005,0.345967,0.000000,6.0,8.0,11.0,0.166667,100.822853,1.564140,0.085291
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,0.872487,0.201985,0.237587,0.0,0.0,6.0,0.333333,99.586327,1.661382,0.043063
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,0.986272,0.141559,1.195703,1.0,2.0,11.0,0.181818,100.278549,1.275520,0.098400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,oportunidad!! se vende amplia casa en col. moc...,oportunidad!! ideal para oficina o casa habita...,Casa,Oriente 172 # 265,Venustiano Carranza,Distrito Federal,20.0,4.0,3.0,...,1.089646,0.136167,2.391406,1.0,1.0,11.0,0.555556,100.875488,1.257188,0.096145
59996,171847,colinas de ecatepec,"casa, sala comedor, patio de servicio, buenas ...",Casa,colinas,Ecatepec de Morelos,Edo. de México,10.0,3.0,1.0,...,1.108382,0.138660,1.195703,0.0,2.0,3.0,0.000000,100.729576,1.367521,0.089020
59997,138313,estrene hermosa casa en sierra morena,hermosa casa lista para habitarse ubicada en f...,Casa,s/calle,Guadalupe,Nuevo León,5.0,3.0,2.0,...,1.323161,0.132296,0.597851,0.0,2.0,8.0,0.285714,100.493149,1.391244,0.054313
59998,271268,zen house i venta de linda casa con acabados ...,hermosa casa con acabados de lujo en fracciona...,Casa,Zen House l,Querétaro,Querétaro,0.0,2.0,1.0,...,0.743564,0.116751,0.000000,2.0,0.0,9.0,0.166667,102.328796,1.597779,0.014146


In [18]:
test['valor_zona'] = test.apply(lambda x: info_ids.get(x['ciudad'], avg), axis = 1)

In [19]:
test

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra,antiguedad_sobre_tipodepropiedad_mean_antiguedad,direccion_cantidad_al_menos_una_mayuscula,direccion_cantidad_fijos_top_k,titulo_cantidad_fijos_top_k,titulo_palabras_top_k_sobre_total_palabras,ciudad_distancia_al_origen,ciudad_mean_mean_todas,ciudad_mean_antiguedad_sobre_mean_metrocubiertos,valor_zona
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,2.0,...,0.105755,3.467539,2.0,4.0,9.0,0.222222,100.743263,1.431429,0.065100,152249.273718
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,0.0,1.0,1.0,...,0.284267,0.000000,0.0,0.0,6.0,0.333333,91.761078,1.654162,0.009316,118528.229270
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,0.345967,0.000000,6.0,8.0,11.0,0.166667,100.822853,1.564140,0.085291,47985.947156
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,0.201985,0.237587,0.0,0.0,6.0,0.333333,99.586327,1.661382,0.043063,50380.130693
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,0.141559,1.195703,1.0,2.0,11.0,0.181818,100.278549,1.275520,0.098400,77646.628364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,oportunidad!! se vende amplia casa en col. moc...,oportunidad!! ideal para oficina o casa habita...,Casa,Oriente 172 # 265,Venustiano Carranza,Distrito Federal,20.0,4.0,3.0,...,0.136167,2.391406,1.0,1.0,11.0,0.555556,100.875488,1.257188,0.096145,71734.802276
59996,171847,colinas de ecatepec,"casa, sala comedor, patio de servicio, buenas ...",Casa,colinas,Ecatepec de Morelos,Edo. de México,10.0,3.0,1.0,...,0.138660,1.195703,0.0,2.0,3.0,0.000000,100.729576,1.367521,0.089020,120297.686165
59997,138313,estrene hermosa casa en sierra morena,hermosa casa lista para habitarse ubicada en f...,Casa,s/calle,Guadalupe,Nuevo León,5.0,3.0,2.0,...,0.132296,0.597851,0.0,2.0,8.0,0.285714,100.493149,1.391244,0.054313,86211.943054
59998,271268,zen house i venta de linda casa con acabados ...,hermosa casa con acabados de lujo en fracciona...,Casa,Zen House l,Querétaro,Querétaro,0.0,2.0,1.0,...,0.116751,0.000000,2.0,0.0,9.0,0.166667,102.328796,1.597779,0.014146,512726.693942


In [22]:
test.to_csv("sets_de_datos/juani_test_XGB2.csv")
train2.to_csv("sets_de_datos/juani_train_XGB2.csv")